In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
pkg_path = "/nfs/homedirs/yuny/project-1/audio_classification"
if pkg_path not in sys.path:
    sys.path.append(pkg_path)
    
import yaml
from audio_classification.tools import do_train

Training on device cuda.


In [2]:
with open("/nfs/homedirs/yuny/project-1/audio_classification/configs/crnn_bmw.yaml", "r") as config_file:
    configs = yaml.load(config_file)

In [ ]:
# Use a loop to train with k fold_cv
base_save_name = configs['CHECKPOINT']['SAVE_NAME']
for i in range(1, 11): # Training using validation fold 1, 2, ...., 10
    print("Start training with validation fold {}".format(i))
    configs['DATASET']['VAL_FOLD']=i
    configs['CHECKPOINT']['SAVE_NAME'] = base_save_name + '-fold' + str(i)
    do_train(configs)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Start training with validation fold 1


Set SLURM handle signals.

   | Name        | Type        | Params
---------------------------------------------
0  | conv1       | Conv2d      | 640   
1  | bn1         | BatchNorm2d | 128   
2  | dropout1    | Dropout     | 0     
3  | conv2       | Conv2d      | 73 K  
4  | bn2         | BatchNorm2d | 256   
5  | dropout2    | Dropout     | 0     
6  | conv3       | Conv2d      | 147 K 
7  | bn3         | BatchNorm2d | 256   
8  | dropout3    | Dropout     | 0     
9  | conv4       | Conv2d      | 147 K 
10 | bn4         | BatchNorm2d | 256   
11 | dropout4    | Dropout     | 0     
12 | rnn         | GRU         | 21 K  
13 | dropout_rnn | Dropout     | 0     
14 | linear      | Linear      | 198   


Epoch 25:  78%|███████▊  | 60/77 [00:17<00:04,  3.41it/s, loss=0.247, v_num=11, train_loss_step=0.00933, val_loss=0.182, val_acc=0.984, train_loss_epoch=0.183]


Epoch 26:   0%|          | 0/77 [00:00<?, ?it/s, loss=0.247, v_num=11, train_loss_step=0.00933, val_loss=0.182, val_acc=0.984, train_loss_epoch=0.183]

In [5]:
# aggregate the results
# TODO: currently need to manually enter these results.
import numpy as np
best_val_acc = np.array([0.984, 1.000, 0.934, 0.967, 0.967, 0.967, 0.984, 0.950, 0.983, 0.967])
end_val_acc = np.array([0.984, 1.000, 0.902, 0.951, 0.951, 0.967, 0.902, 0.933, 0.967, 0.95])
best_epoch = np.array([33, 42, 22, 28, 45, 44, 42, 25, 22, 41])

In [11]:
print("The average validation accuracy taking from the best epoch is %.3f (std=%.3f)" 
      %(best_val_acc.mean(), best_val_acc.std()))
print("The average validation accuracy taking from the last epoch is %.3f (std=%.3f)" 
      %(end_val_acc.mean(), end_val_acc.std()))
print("The average best epoch is %.1f (std=%.1f)" %(best_epoch.mean(), best_epoch.std()))

The average validation accuracy taking from the best epoch is 0.970 (std=0.018)
The average validation accuracy taking from the last epoch is 0.951 (std=0.030)
The average best epoch is 34.4 (std=9.0)


In [20]:
# TODO: Temporary testing procedure, will move to do_train in the future
from audio_classification.model import LitCRNN
from audio_classification.data import BMWDataset
from audio_classification.tools.train_net import collate, get_transform
import pytorch_lightning as pl
from torch.utils.data import DataLoader

test_set = BMWDataset(configs, [11], transform=get_transform(configs))
print("Test set contains {} samples".format(len(test_set)))
test_loader = DataLoader(test_set, batch_size=configs["DATALOADER"]["BATCH_SIZE"],
                                num_workers=configs["DATALOADER"]["NUM_WORKERS"],
                                pin_memory=True, collate_fn = collate)


map_location = 'cuda'
model = LitCRNN.load_from_checkpoint(
    checkpoint_path='/nfs/homedirs/yuny/project-1/weights/bmw_week3/crnn-bmw-fold3-epoch=22-val_acc=0.934.ckpt',
    cfg=configs,
    map_location=map_location
)
trainer = pl.Trainer()

trainer.test(model, test_dataloaders=test_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
Set SLURM handle signals.


Test set contains 60 samples
Testing: 100%|██████████| 8/8 [00:28<00:00,  2.97s/it]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9167), 'test_loss': tensor(0.2556)}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 8/8 [00:29<00:00,  3.67s/it]


[{'test_loss': 0.2555873990058899, 'test_acc': 0.9166666865348816}]